In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from numba import jit
import random

# @jit(nopython=True)
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [5]:
%time monte_carlo_pi(10000)

Wall time: 5.99 ms


3.1196

In [6]:
monte_carlo_pi_jit = jit()(monte_carlo_pi)

In [7]:
%time monte_carlo_pi_jit(10000)  # first time run - jit - has to compile it, gonna take longer, code has been cached

Wall time: 744 ms


3.11

In [8]:
%time monte_carlo_pi_jit(10000)

Wall time: 0 ns


3.1568

In [9]:
from numba import jit, njit, vectorize

In [10]:
def original_fct(input_list):
    output_list = []
    for item in input_list:
        if item % 2 == 0:
            output_list.append(2) # number assumed it represents n64
        else:
            output_list.append('1') # here it becomes confused
    return output_list

test_array = list(range(10000))

In [12]:
%time _ = original_fct(test_array)

Wall time: 981 µs


In [13]:
jitted_function = jit()(original_fct)

In [14]:
%time _ = jitted_function(test_array) # warning, not an error

C:\Users\alexb\AppData\Local\Temp/ipykernel_7980/4232965370.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "original_fct" failed type inference due to: Invalid use of BoundFunction(list.append for list(int64)<iv=None>) with parameters (Literal[str](1))

During: resolving callee type: BoundFunction(list.append for list(int64)<iv=None>)
During: typing of call at C:\Users\alexb\AppData\Local\Temp/ipykernel_7980/4232965370.py (7)


File "C:\Users\alexb\AppData\Local\Temp\ipykernel_7980\4232965370.py", line 7:
<source missing, REPL/exec in use?>

  def original_fct(input_list):
C:\Users\alexb\AppData\Local\Temp/ipykernel_7980/4232965370.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "original_fct" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "C:\Users\alexb\AppData\Local\Temp\ipykernel_7980\4232965370.py", l

Wall time: 412 ms


c:\users\alexb\appdata\local\programs\python\python39\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "original_fct" was compiled in object mode without forceobj=True.

File "C:\Users\alexb\AppData\Local\Temp\ipykernel_7980\4232965370.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
c:\users\alexb\appdata\local\programs\python\python39\lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "C:\Users\alexb\AppData\Local\Temp\ipykernel_7980\4232965370.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaDeprecationWarning(msg,


In [15]:
%time _ = jitted_function(test_array)

Wall time: 3 ms


In [16]:
jitted_function = jit(nopython=True)(original_fct)  # now bcs nopython=True, we get an error, not a warning

In [17]:
%time _ = jitted_function(test_array)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of BoundFunction(list.append for list(int64)<iv=None>) with parameters (Literal[str](1))
[0m
[0m[1mDuring: resolving callee type: BoundFunction(list.append for list(int64)<iv=None>)[0m
[0m[1mDuring: typing of call at C:\Users\alexb\AppData\Local\Temp/ipykernel_7980/4232965370.py (7)
[0m
[1m
File "C:\Users\alexb\AppData\Local\Temp\ipykernel_7980\4232965370.py", line 7:[0m
[1m<source missing, REPL/exec in use?>[0m


In [18]:
jitted_function = njit()(original_fct)  # njit decorator automatically says nopython=True

In [31]:
# fixing the code
def original_fct(input_list):
    output_list = []
    for item in input_list:
        if item % 2 == 0:
            output_list.append(2) # number assumed it represents n64
        else:
            output_list.append(1) # here it becomes confused
    return output_list

# test_array = list(range(10000))
test_array = np.arange(10000)

In [32]:
jitted_function = njit()(original_fct)  # njit decorator automatically says nopython=True

In [34]:
%time _ = jitted_function(test_array)  # passing a python list, numba has to use reflection...

Wall time: 0 ns


In [38]:
@vectorize
def scalar_computation(num):
    if num % 2 ==0:
        return 2
    else:
        return 1

# different structure of the fct...

In [39]:
%time scalar_computation(test_array)

Wall time: 103 ms


array([2, 1, 2, ..., 1, 2, 1], dtype=int64)

In [41]:
@njit
def fixed_fct(input_list):
    output_list = np.zeros_like(input_list) # faster bcs it knows its dimension; prealocated
    for ii,item in enumerate(input_list):
        if item % 2 == 0:
            output_list[ii] = (2)
        else:
            output_list[ii] = (1)
    
    return output_list
            

In [45]:
%time fixed_fct(test_array)

Wall time: 0 ns


array([2, 1, 2, ..., 1, 2, 1])